In [1]:
# Here we take care of paths.

from pathlib import Path
import os
print('Starting path:' + os.getcwd())
if os.getcwd()[-16:] == 'image-to-tex-OCR':
    pass
else:
    PATH = Path().resolve().parents[0]
    os.chdir(PATH)

# make sure you are in Paragraph_to_Tex folder
print('Current path:' + os.getcwd())

Starting path:/Users/gregory/PROJECT_ML/PROJECT_AMINE/image-to-tex-OCR/Jupyter_Notebooks
Current path:/Users/gregory/PROJECT_ML/PROJECT_AMINE/image-to-tex-OCR


In [2]:
# Import the Printed_Tex_Data_Module

from Data.Data_Module import Data_Module
import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image
import torch
from torchvision import transforms
from IPython.display import display, Math

In [4]:
data = Data_Module(stage = 'fit',
                 set_max_label_length = 256,
                 number_png_images_to_use_in_dataset=200*1000,
                 labels_transform='default',
                image_transform_name ='alb',
                 train_val_fraction = 0.95,
                   
                load_vocabulary = False,


                 batch_size = 64,
                num_workers = 10,
                data_on_gpu = False,
                )



# Generates the dataframe from the images and formulas
# images need to be in the folder Data/Data_Bank/generated_png_images
# formulas need to be in Data/Data_Bank/final_png_formulas.txt
# image filenames need to be in Data/Data_Bank/corresponding_png_images.txt
data.prepare_data()

# if one wants to load a vocabulary do not use data.prepare_dataframe(), 
# but instead set load_vocabulary = True


In [5]:
data.df.head(5)

,formula,image_name,height,width,formula_length,latex_tokenized,tokenized_len
0,\widehat { H } = \displaystyle \frac { 2 \pi }...,db69228953de400.png,64,512,289,"[\widehat, {, H, }, =, \displaystyle, \frac, {...",83
1,D _ { \alpha } ^ { i } D ^ { j \alpha } \; w =...,a452f8cc9263922.png,64,512,135,"[D, _, {, \alpha, }, ^, {, i, }, D, ^, {, j, \...",52
2,s h _ { 2 } + \nu = \partial ^ { i } \left( 2 ...,b76614553695957.png,64,512,303,"[s, h, _, {, 2, }, +, \nu, =, \partial, ^, {, ...",104
3,Q _ { 2 n } ( { \bf A } ) = \left\langle { \bf...,56102e6fec5ead8.png,64,512,74,"[Q, _, {, 2, n, }, (, {, \bf, A, }, ), =, \lef...",24
4,Z _ { 2 n } = 1 - \frac { 1 } { n } \ell \delt...,efbd72c272f1c47.png,64,512,199,"[Z, _, {, 2, n, }, =, 1, -, \frac, {, 1, }, {,...",76


In [8]:
print( 
    'Vocabulary size:',len(data.vocabulary),
    '\nMax label length:', data.max_label_length,
    "\nStart <S> goes to index ",data.vocabulary['<S>'],
      "\nEnd <E> goes to index ",data.vocabulary['<E>'],
      "\nPadding <P> goes to index ",data.vocabulary['<P>'],
)

Vocabulary size: 179 
Max label length: 258 
Start <S> goes to index  0 
End <E> goes to index  1 
Padding <P> goes to index  2


In [7]:
# prepeares the dataloaders with transformations and splits train/val
data.setup()

Train/Val Data is ready for Model loading.


In [10]:
data.inverse_vocabulary

{0: '<S>',
 1: '<E>',
 2: '<P>',
 3: '!',
 4: '&',
 5: '(',
 6: ')',
 7: '+',
 8: ',',
 9: '-',
 10: '--',
 11: '.',
 12: '/',
 13: '0',
 14: '1',
 15: '2',
 16: '3',
 17: '4',
 18: '5',
 19: '6',
 20: '8',
 21: ':',
 22: ';',
 23: '=',
 24: '>',
 25: 'A',
 26: 'B',
 27: 'C',
 28: 'D',
 29: 'E',
 30: 'F',
 31: 'G',
 32: 'H',
 33: 'I',
 34: 'K',
 35: 'L',
 36: 'M',
 37: 'N',
 38: 'O',
 39: 'P',
 40: 'Q',
 41: 'R',
 42: 'S',
 43: 'T',
 44: 'U',
 45: 'V',
 46: 'W',
 47: 'X',
 48: 'Y',
 49: 'Z',
 50: '[',
 51: '\\',
 52: '\\,',
 53: '\\:',
 54: '\\;',
 55: '\\Big',
 56: '\\Gamma',
 57: '\\Lambda',
 58: '\\Omega',
 59: '\\Phi',
 60: '\\\\',
 61: '\\aleph',
 62: '\\alpha',
 63: '\\approx',
 64: '\\bar',
 65: '\\begin{array}',
 66: '\\begin{matrix}',
 67: '\\beta',
 68: '\\bf',
 69: '\\cal',
 70: '\\cdot',
 71: '\\cdots',
 72: '\\cong',
 73: '\\dagger',
 74: '\\delta',
 75: '\\displaystyle',
 76: '\\dot',
 77: '\\dots',
 78: '\\ell',
 79: '\\end{array}',
 80: '\\end{matrix}',
 81: '\\epsilon'

# Checking the data 


In [ ]:
data.df['image_name'][10]

In [ ]:
# How to access generated png images 
IMAGE_NAME = data.df['image_name'][10]
image_path= "Data/Data_Bank/generated_png_images/" + IMAGE_NAME
image = Image.open(image_path)#.convert("L")
print('Image is below')
display(image)

# Related Label

image_label_list = data.df[data.df['image_name'] ==  IMAGE_NAME]['formula'].tolist()#.item()
image_label = " ".join(image_label_list)
print('Label:')
display(Math(image_label))
print('Tex formula: \n')
print(data.df[data.df['image_name'] == IMAGE_NAME]['formula'].item())

### Check  vocabulary 

In [ ]:
print(data.vocabulary)

In [ ]:
# Can acrtually see the frequncies

data.data_server.vocabulary_dataframe.sort_values("freq", ascending=False)

In [ ]:
# Plot of vocabulary
import matplotlib.pyplot as plt

# first order the columns by decsending frequncy 
data.data_server.vocabulary_dataframe = data.data_server.vocabulary_dataframe.sort_values("freq", ascending=False)

# next we plot them
data.data_server.vocabulary_dataframe.plot(x="token", y='freq', kind='barh', figsize =(16,75))
plt.show()

#### Look at the distribution of data

In [ ]:
# We plot the tokenized len distribution

tokenized_len = data.df['tokenized_len'].value_counts()
tokenized_len

In [ ]:
# How long are formulas
plt.figure(figsize=(40,5))
sns.countplot(x=data.df['tokenized_len']);

## Acessing the data

In [ ]:
# Helper Function to convert prediction labels to strings
def token_to_strings(tokens):
    mapping = data.vocabulary
    inverse_mapping =data.inverse_vocabulary
    s=''
    if tokens.shape[0] ==1:
        tokens = tokens[0]
    for number in tokens:
        letter = inverse_mapping[number.item()]
        s= s + str(letter)
    return s


In [ ]:
transform = transforms.ToPILImage()
display(transform(data.data_train[1][0]))
print(token_to_strings(data.data_train[1][1]))